In [1]:
import shapefile as shp
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [7]:
sf = shp.Reader(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\ZCTA_US_shapefiles\cb_2018_us_zcta510_500k.shp')

In [8]:
records = sf.shapeRecords()

In [9]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['ZCTA5CE10', 'C', 5, 0],
 ['AFFGEOID10', 'C', 14, 0],
 ['GEOID10', 'C', 5, 0],
 ['ALAND10', 'N', 14, 0],
 ['AWATER10', 'N', 14, 0]]

In [10]:
tract_file = 'ZCTA_New_York_logt'
df = pd.read_csv(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\{}.csv'.format(tract_file))
geojsons = []
used_tract_ids = set()

In [11]:
for record in tqdm(records):
    tract_id = int(record.record[0])
    try:
        row = df.loc[df['ZCTA5'] == tract_id].iloc[0]
    except IndexError:
        continue
    used_tract_ids.add(tract_id)
    d = record.__geo_interface__
    d['properties'] = {'id': tract_id}
    for c in df.columns:
        if c.endswith('_CrudePrev'):
            d['properties'][c] = row[c]
    for c in ('education_closest_travel_times', 'food_closest_travel_times', 'worship_closest_travel_times',
              'physical_closest_dist', 'transport_closest_dist'):
        d['properties'][c] = row[c]
    geojsons.append(d)

100%|██████████████████████████████████████████████████████████████████████████| 33144/33144 [00:17<00:00, 1882.12it/s]


In [12]:
#sf2 = shp.Reader(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\census_tract_New_York\cb_2018_36_tract_500k.shp')

In [13]:
#records2 = sf2.shapeRecords()

In [14]:
# for record in tqdm(records2):
#     tract_id = int(record.record[4])
#     if tract_id in used_tract_ids:
#         continue
#     try:
#         row = df.loc[df['TractFIPS'] == tract_id].iloc[0]
#     except IndexError:
#         continue
#     used_tract_ids.add(tract_id)
#     d = record.__geo_interface__
#     d['properties'] = {'id': tract_id}
#     for c in df.columns:
#         if c.endswith('_CrudePrev'):
#             d['properties'][c] = row[c]
#     for c in ('education_closest_travel_times', 'food_closest_travel_times', 'worship_closest_travel_times',
#               'physical_closest_dist', 'transport_closest_dist', 'zcta_x', 'zcta_y'):
#         d['properties'][c] = row[c]
#     geojsons.append(d)

In [15]:
len(geojsons)

184

In [16]:
geojsons = {"type":"FeatureCollection", "features": geojsons}

In [17]:
with open(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\{}.json'.format(tract_file), 'w') as f:
    json.dump(geojsons, f)

In [22]:
# val_mins = {}
# val_maxes = {}
# for f in geojsons['features']:
#     for k, v in f['properties'].items():
#         if k.endswith('CrudePrev'):
#             try:
#                 if v < val_mins[k]:
#                     val_mins[k] = v
#             except KeyError:
#                 val_mins[k] = v
#             try:
#                 if v > val_maxes[k]:
#                     val_maxes[k] = v
#             except KeyError:
#                 val_maxes[k] = v
# with open(r'C:\Users\willd\Documents\Georgia Tech\CSE6424\Project\mins_maxes.json', 'w') as f:
#     json.dump({k: (v, val_maxes[k]) for k, v in val_mins.items()}, f)